# Genetische Algorithmen

#### Patrick Schnider, Departement Mathematik und Informatik, Universität Basel 

In dieser Fallstudie werden wir eine Strategie kennenlernen, um schwierige Probleme zu lösen. Der Ansatz ist einfach: Wir probieren verschiedene Lösungen aus, und kombinieren die vielversprechendsten davon zu neuen, hoffentlich besseren Lösungen. 

Die Strategie, die wir hier anwenden ist unter dem Namen *Genetische Algorithmen* bekannt, da diese einen evolutionären Prozess simulieren.  

Genetische Algorithmen sind auch eine Anwendung, bei der wir extensiv mit Listen, Listen von Tupeln, und Listen von Listen arbeiten müssen. 

### Problemstellung: Knapsack Problem

Als praktisches Problem, das wir mit genetischen Algorithmen lösen wollen, betrachten wir das Knapsack-Problem. Dies ist ein klassiches Problem in der Informatik. Die Aufgabe ist es, einen Rucksack so zu packen, dass die darin enthaltenen Gegenstände einen möglichst grossen Nutzen bringen. Der gepackte Rucksack darf aber ein bestimmtes Maximalgewicht nicht überschreiten. 

Als Beispiel berachten wir folgende Gegenstände:

| Gegenstand | Gewicht | Nutzen | 
|------------|---------|--------|
| Schlafsack | 15      |  15    |
| Seil       | 3 | 7 |
| Messer | 2 | 10 | 
| Taschenlampe | 5 | 5|
| Flasche | 9 | 8 | 
| Essen | 20 | 17|





#### Aufgabe

* Wie können wir so eine Tabelle in Python abbilden?

Diese Liste können wir als Liste von Tupeln abbilden:

In [1]:
packing_list = [
    ("Schlafsack", 15, 15),
    ("Seil", 3, 7),
    ("Messer", 2, 10),
    ("Taschenlampe", 5, 5),
    ("Flasche", 9, 8),
    ("Essen", 20, 17)
]

# Das erlaubte Maximalgewicht
max_weight = 30


### Gene zur Repräsentation der Lösung

Die Idee hinter genetischen Algorithmen ist jetzt, dass wir eine mögliche Lösung als Liste von 0 und 1 abbilden. Wenn an der ersten Stelle eine 1 steht, dann nehmen wir den Entsprechenden Gegenstand mit. Wenn eine 0 steht, wird der Gegenstand zuhause gelassen. Die Liste

```
[0, 1, 0, 0, 1, 0] 
```
repräsentiert eine Lösung, bei der wir das Seil und die Flasche mitnehmen. 

Eine solche Liste wird, in Anlehnung an die Genetik,  Chromosom genannt. 

##### Aufgabe 

Schreiben Sie eine Funktion, die ein zufälliges Chromosom der richtigen Länge generiert. Nutzen Sie dazu die Funktion `random.randint`, die Sie schon früher kennengelernt haben. Zur Erinnerung:
```
random.randint(0, n)
```
erzeugt zufällige Ganzzahlen im Interval $[0, n]$.


In [2]:
import random
def random_chromosome():
    chromosome = []
    for i in range(0, len(packing_list)):
        chromosome.append(random.randint(0, 1))
    return chromosome


Wir schreiben nun eine Funktion, die ein Chromosom entgegennimmt und eine Liste mit den Gegenständen ausgibt, die in der Lösung enthalten sind. 

In [ ]:
def packed_items(chromosome):
    item_list = []
    for i in range(0, len(chromosome)):
        if chromosome[i] == 1:
            item_list.append(packing_list[i][0])
    return item_list

print(packed_items(random_chromosome()))

Zudem brauchen wir noch eine Funktion, die uns eine zufällige Population von Chromosomen erzeugt. 

In [5]:
def random_population(n):
    population = []
    for i in range(0, n):
        chromosome = random_chromosome()
        population.append(chromosome)
    return population


### Fitnessfunktion

Nun können wir eine Fitnessfunktion schreiben. Die Fitnessfunktion gibt uns für jedes Chromosom (also jede Lösung) einen Fitnesswert. Dieser entspricht einfach dem aufsummierten Nutzen. Wenn das Gewicht über dem erlaubten Totalgewicht liegt, ist die Fitness 0. 


#### Aufgabe

Implementieren und testen Sie die Fitnessfunktion 

In [6]:
def fitness(chromosome):
    total_points = 0
    total_weight = 0

    for i in range(0, len(chromosome)):
        (product, weight, point) = packing_list[i]
        if chromosome[i] == 1:
            total_points = total_points + point
            total_weight = total_weight + weight
    if total_weight > max_weight:
        return 0
    else:
        return total_points


In [ ]:
print(fitness([0, 1, 0, 1, 1, 1]))
print(fitness([1, 0, 0, 0, 0, 0]))

### Genetische Algorithmen

Genetische Algorithmen führen nun eine Simulation durch, in der eine Population von Chromosomen über Generationen entwickelt wird. Dabei kommt folgender Mechanismus zum tragen:

1. Fittere Chromosomen werden mit höherer Wahrscheinlichkeit ausgewählt
2. Diversität in der Population wird gewahrt, indem eine Art Fortpflanzung (cross_over) stattfindet, bei der die Chromosomen gemischt werden. 
3. Es werden zufällige Mutationen eingeführt. 

#### Selektion von guten Chromosomen

Die folgende Funktion selektiert zufällig Chromosomen aus der Population. Dabei ist die Wahrscheinlichkeit eines Chromosoms selektiert zu werden proportional zur Fitness.

*Hinweis: Sie müssen diesen Algorithmus nicht im Detail verstehen. Dies ist auch manchmal in der Praxis so. Sie müssen die Funktion aber anwenden können*

In [9]:
import random
def select_surviving(population):
    roulette_wheel = []
    for chromosome in population:
        fitness_of_chromosome = fitness(chromosome)

        for i in range(0, fitness_of_chromosome):
            roulette_wheel.append(chromosome)

    new_population = []
    for i in range(0, len(population)):
        r = random.randint(0, len(roulette_wheel) - 1)
        new_population.append(roulette_wheel[r])
        
    return new_population
    


Wir können nun eine Simulation starten. 

In [ ]:
def simulate(num_initial_genes, num_generations):
    
    population = random_population(num_initial_genes)    
    for i in range(0, num_generations):
        population = select_surviving(population)
        i = i + 1
    return population

population = simulate(10, 100)

for chromosome in population:
    print("chromosome ", packed_items(chromosome), "fitness ", fitness(chromosome))

#### Aufgabe:

* Geben Sie die Population nach 100 Generationen aus. Wie sieht dies aus?

Um die Diversität zu erhalten, können wir eine zufällige Mutation auf jedem Chromosomen einführen

In [32]:
def mutate(chromosome):
    new_chromosome = []
    r = random.randint(0, len(chromosome) - 1)
    for i in range(0, len(chromosome)):
        if i != r:
            new_chromosome.append(chromosome[i])
        else:
            new_chromosome.append(1 - chromosome[i])
    return new_chromosome

In der Simulation mutieren wir dann jedes Chromosome mit einer bestimmten Wahrscheinlichkeit. 

In [ ]:
def simulate(num_initial_genes, num_generations):
    
    mutation_probability = 0.5
    
    population = random_population(num_initial_genes)    
    for i in range(0, num_generations):
        population = select_surviving(population)
        for j in range(0, len(population)):
            
            if random.randint(0, 100) < mutation_probability * 100:
                population[j] = mutate(population[j])
        i = i + 1
    return population

population = simulate(10, 100)

for gene in population:
    print("gene ", packed_items(gene), "fitness ", fitness(gene))

Eine weitere Möglichkeit die Diversität zu erhalten ist, dass wir Chromosomen mischen. Der erste Teil des Chromosomens wird von einem Elternteil genommen, der Zweite Teil vom anderen. Der Punkt, an welchem die Chromosomen geteilt werden, wird zufällig ausgewürfelt. In folgendem Beispiel ist dies an Position 4. 

Beispiel

| | | 
|----------|----------|
| Parent 1 | Parent 2 | 
| 00100101 |  11011110 |  
| Child 1  | Child 2 | 
| 00101110 | 11010101 |

Die crossover Funktionalität kann wie folgt implementiert werden.

In [38]:
def cross_over(chromosome1, chromosome2):
    
    crossover_point = random.randint(1, len(chromosome1))
    child1 = []
    child2 = []
    for i in range(0, crossover_point):
        child1.append(chromosome1[i])
        child2.append(chromosome2[i])

    for i in range(crossover_point, len(chromosome1)):
        child1.append(chromosome2[i])
        child2.append(chromosome1[i])
    return (child1, child2)


Nun haben wir alle Bausteile zusammen, um die finale Simulation zu implementieren

In [39]:
def simulate(num_initial_genes, num_generations):
    
    mutation_probability = 0.01
    crossover_probability = 0.1

    population = random_population(num_initial_genes)
    parents = select_surviving(population) 
    
    for i in range(0, num_generations):        
        for j in range(0, len(parents) // 2):
            if random.randint(0, 100) < crossover_probability * 100:
                (child1, child2) = cross_over(parents[2 * j], parents[2 * j + 1])
            else:
                (child1, child2) = (parents[2 * j], parents[2 * j + 1])
                
            if random.randint(0, 100) < mutation_probability * 100:
                child1 = mutate(child1)
            if random.randint(0, 100) < mutation_probability * 100:
                child2 = mutate(child2)
            
            population[2 * j] = child1
            population[2 * j + 1] = child2
             
        parents = select_surviving(population)   


        i = i + 1
    return parents

In [ ]:
population = simulate(50, 100)
for gene in population:
    print("gene ", packed_items(gene), "fitness ", fitness(gene))

### Aufgabe 

* Experimentieren Sie mit dieser Simulation. Wie gut ist die Lösung? Bekommen Sie jedes Mal die optimale Lösung?
